# C9orf72 age at onset analysis
This notebook contains the source code to perform the age at onset analysis of the c9orf72 manuscript. The source data and analysis exist in the original folders but I want to create a folder to keep the essential data and code for Github and future references.

## Individual variant age at onset regression

In [5]:
%%bash
cd /data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/

# I am  trying to identify the SNPs that contribute the most to GRS. I regressed the genotype againts age at onset for every and each snp in decile 10.
# Additionally, I split my data in training and replication to asses reproducibility

module load R/4.0.0
R --vanilla --no-save
require(dplyr)
require(tidyverse)
require(ggplot2)
require(data.table)
require(RColorBrewer)
library(caTools)


profile= read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/Merged.162.RISK.profile", header = T)
dim(profile)
profile$FID_IID = profile$FID
#PHENO FILE 
pheno = read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2021-03-26.MergingDatasets/PCs/UPDATED.COV.ALSplusALSFTD.C9.txt", header = T)
dim(pheno)


#merge pheno with profile
data = merge(profile,pheno, by= "FID_IID")


table(data$diagnosis3)
table(data$diagnosis2)
#Case_EXP, Case_WT, Control_EXP, Control_WT
table(data$diagnosis4)

dim(data)
data$PHENO = data$PHENO.x

# REGRESSION C9ORF72CARRIERS using GRS without C9orf72
data <- data %>% mutate (diagnosis5 = ifelse(diagnosis2 == "Case_EXP", "Case_EXP", 
                                                        ifelse(diagnosis2 == "Case_WT", "Case_WT",
                                                               ifelse(diagnosis3 == "Control", "Control","no"))))


data3groups = filter(data, diagnosis5 == "Case_EXP" |diagnosis5 == "Case_WT" | diagnosis5 == "Control")
dim(data3groups)

data3groups = filter(data3groups, age_at_onset > 1)
dim(data3groups)

# Whole dataset

# open raw data file
raw = fread("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Bin/162variants.Merged.Discovery.raw", header = T)
# open raw data file
data3groups$FID = data3groups$FID.x

data = merge(raw, data3groups, by = "FID")



summary = data %>% group_by(diagnosis2) %>% 
  summarize(mean_AAO = mean(age_at_onset), 
            SD_AAO = sd(age_at_onset), count = n())
summary = as.data.table(summary)
print(summary)



######## Additive model (effect of each additional allele) ###################

dataexp = filter(data, diagnosis2 == "Case_EXP")
dim(dataexp)
table(dataexp$dataset)




AAO.merged.C9 <- lm(age_at_onset ~ rs9901522_T  + GENDER  + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = dataexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared


AAO.merged.C9 <- lm(age_at_onset ~ rs113247976_T  + GENDER + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = dataexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared



datnoaexp = filter(data, diagnosis2 == "Case_WT")





AAO.nonC9 <- lm(age_at_onset ~ rs9901522_T   + GENDER  + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = datnoaexp)
summary(AAO.nonC9)
summary(AAO.nonC9)$adj.r.squared

AAO.nonC9 <- lm(age_at_onset ~ rs113247976_T  + GENDER  + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = datnoaexp)
summary(AAO.nonC9)
summary(AAO.nonC9)$adj.r.squared




######## Dominant model (presence versus absence of minor allele) #############




dataexp <- dataexp %>% mutate (rs9901522_T.dominant = ifelse(rs9901522_T == "0", "0", 
                                                        ifelse(rs9901522_T == "1", "1",
                                                               ifelse(rs9901522_T ==  "2", "1","no"))))



dataexp <- dataexp %>% mutate (rs113247976_T.dominant = ifelse(rs113247976_T == "0", "0", 
                                                        ifelse(rs113247976_T == "1", "1",
                                                               ifelse(rs113247976_T ==  "2", "1","no"))))




AAO.merged.C9 <- lm(age_at_onset ~ rs9901522_T.dominant  + GENDER + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = dataexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared

AAO.merged.C9 <- lm(age_at_onset ~ rs113247976_T.dominant  + GENDER  + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = dataexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared



datnoaexp = filter(data, diagnosis2 == "Case_WT")


datnoaexp <- datnoaexp %>% mutate (rs9901522_T.dominant = ifelse(rs9901522_T == "0", "0", 
                                                        ifelse(rs9901522_T == "1", "1",
                                                               ifelse(rs9901522_T ==  "2", "1","no"))))



datnoaexp <- datnoaexp %>% mutate (rs113247976_T.dominant = ifelse(rs113247976_T == "0", "0", 
                                                        ifelse(rs113247976_T == "1", "1",
                                                               ifelse(rs113247976_T ==  "2", "1","no"))))




AAO.merged.C9 <- lm(age_at_onset ~ rs9901522_T.dominant  + GENDER + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = datnoaexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared

AAO.merged.C9 <- lm(age_at_onset ~ rs113247976_T.dominant  + GENDER  + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20, data = datnoaexp)
summary(AAO.merged.C9)
summary(AAO.merged.C9)$adj.r.squared

# recesive model: presence versus absence of 2 minor alleles
# for kif5a we dont have 2 minor alleles genotype and only 4 for the other one, so it is pointless to do it.







[+] Loading gcc  9.2.0  ... 
[+] Loading GSL 2.6 for GCC 9.2.0 ...
[-] Unloading gcc  9.2.0  ... 
[+] Loading gcc  9.2.0  ... 
[+] Loading openmpi 3.1.4  for GCC 9.2.0 
[+] Loading ImageMagick  7.0.8  on cn1030 
[+] Loading HDF5  1.10.4 
[-] Unloading gcc  9.2.0  ... 
[+] Loading gcc  9.2.0  ... 
[+] Loading NetCDF 4.7.4_gcc9.2.0 
[+] Loading pandoc  2.16.2  on cn1030 
[+] Loading pcre2 10.21  ... 
[+] Loading R 4.0.0 



R version 4.0.0 (2020-04-24) -- "Arbor Day"
Copyright (C) 2020 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> require(dplyr)


Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
package ‘dplyr’ was built under R version 4.0.3 


> require(tidyverse)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.1.4     ✔ forcats 0.5.0
✔ readr   2.1.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning messages:
1: package ‘ggplot2’ was built under R version 4.0.3 
2: package ‘tibble’ was built under R version 4.0.3 
3: package ‘tidyr’ was built under R version 4.0.3 
4: package ‘readr’ was built under R version 4.0.3 


> require(ggplot2)
> require(data.table)


Loading required package: data.table

Attaching package: ‘data.table’

The following object is masked from ‘package:purrr’:

    transpose

The following objects are masked from ‘package:dplyr’:

    between, first, last

Warning message:
package ‘data.table’ was built under R version 4.0.3 
Loading required package: RColorBrewer


> require(RColorBrewer)
> library(caTools)
> 
> 
> profile= read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/Merged.162.RISK.profile", header = T)
> dim(profile)
[1] 47184     6
> profile$FID_IID = profile$FID
> #PHENO FILE 
> pheno = read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2021-03-26.MergingDatasets/PCs/UPDATED.COV.ALSplusALSFTD.C9.txt", header = T)
> dim(pheno)
[1] 42090    38
> 
> 
> #merge pheno with profile
> data = merge(profile,pheno, by= "FID_IID")
> 
> 
> table(data$diagnosis3)

   Case Control 
   7854   34236 
> table(data$diagnosis2)

   Case_EXP     Case_WT Control_EXP  Control_WT 
        817        7037           1       34235 
> #Case_EXP, Case_WT, Control_EXP, Control_WT
> table(data$diagnosis4)

    ALS Control     FTD 
   7703   34236     151 
> 
> dim(data)
[1] 42090    44
> data$PHENO = data$PHENO.x
> 
> # REGRESSION C9ORF72CARRIERS using GRS without C9orf72
> data <- data %>% mutate (diagnosis5 = ifelse(diagnosis2 == "Cas

### Individual variants age at onset plots

In [2]:
%%bash


module load R/4.0.0
R --vanilla --no-save
require(dplyr)
require(tidyverse)
require(ggplot2)
require(data.table)
require(RColorBrewer)
setwd("Merged.toPLOT.LOCAL.2021-04-11.GRS")



############################################# OPEN .PROFILE AND COVARIATES FILES ##################################################

profile= read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/Merged.162.RISK.profile", header = T)
profile$FID_IID = profile$FID
#PHENO FILE 
pheno = read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2021-03-26.MergingDatasets/PCs/UPDATED.COV.ALSplusALSFTD.C9.txt", header = T)
dim(pheno)





#merge pheno with profile
data = merge(profile,pheno, by= "FID_IID")


table(data$diagnosis3)
table(data$diagnosis2)
#Case_EXP, Case_WT, Control_EXP, Control_WT
table(data$diagnosis4)

dim(data)
data$PHENO = data$PHENO.x

# REGRESSION C9ORF72CARRIERS using GRS without C9orf72
data <- data %>% mutate (diagnosis5 = ifelse(diagnosis2 == "Case_EXP", "Case_EXP", 
                                                        ifelse(diagnosis2 == "Case_WT", "Case_WT",
                                                               ifelse(diagnosis3 == "Control", "Control","no"))))


data3groups = filter(data, diagnosis5 == "Case_EXP" |diagnosis5 == "Case_WT" | diagnosis5 == "Control")
dim(data3groups)

data3groups = filter(data3groups, age_at_onset > 1)
dim(data3groups)





# open raw data file
raw = fread("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Bin/162variants.Merged.Discovery.raw", header = T)
# open raw data file
data3groups$FID = data3groups$FID.x

data = merge(raw, data3groups, by = "FID")

data = filter(data, diagnosis2 == "Case_EXP")



data = filter(data, diagnosis2 == "Case_EXP")
table(data$diagnosis2)

p <- ggplot(data, aes(x = as.factor(rs9901522_T), y=age_at_onset, fill=rs9901522_T)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('#354E71', '#354E71', '#354E71'), alpha = 1/4,  colour = c('#354E71', '#354E71', '#354E71')) + 
  theme_classic(base_size = 40) + coord_flip()
 
 p2 = p + ylab("Age at onset") + xlab("rs9901522 Genotype") +
scale_x_discrete(name = "rs9901522 Genotype", breaks=c("0","1", "2"), labels=c("CC","CT", "TT")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")

#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/updated.BoxPlot_rs9901522.Merged.png", width = 40, height = 30, dpi = 300, units = "cm", plot = p2) 


p <- ggplot(data, aes(x = as.factor(rs113247976_T), y=age_at_onset, fill=rs113247976_T)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('#CE4A7EFF', '#CE4A7EFF'), alpha = 1/4,  
  colour = c('#CE4A7EFF', '#CE4A7EFF')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("rs113247976_T Genotype") +
scale_x_discrete(name = "rs113247976 Genotype", breaks=c("0","1"), labels=c("CC","CT")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")


#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/updated.BoxPlot_rs113247976.Merged.png", width = 40, height = 20, dpi = 300, units = "cm", plot = p2) 


p <- ggplot(data, aes(x = diagnosis5, y=age_at_onset, fill=diagnosis5)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('grey24'), alpha = 1/4,  
  colour = c('grey24')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("C9orf72 cases") +
scale_x_discrete(name = "C9orf72 cases", breaks=c("Case_EXP"), labels=c("")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")

#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/updated.BoxPlot_C9orf72cases.Merged.png", width = 40, height = 20, dpi = 300, units = "cm", plot = p2) 




data = merge(raw, data3groups, by = "FID")
data = filter(data, diagnosis2 == "Case_WT")

p <- ggplot(data, aes(x = as.factor(rs9901522_T), y=age_at_onset, fill=rs9901522_T)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('#354E71', '#354E71', '#354E71'), alpha = 1/4,  colour = c('#354E71', '#354E71', '#354E71')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("rs9901522 Genotype") +
scale_x_discrete(name = "rs9901522 Genotype", breaks=c("0","1", "2"), labels=c("CC","CT", "TT")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")


#ggsave(filename = "BoxPlot_rs9901522.Merged.WT.png", width = 40, height = 30, dpi = 300, units = "cm", plot = p2) 

data = merge(raw, data3groups, by = "FID")

data = filter(data, diagnosis2 == "Case_WT")



p <- ggplot(data, aes(x = diagnosis5, y=age_at_onset, fill=diagnosis5)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('grey24'), alpha = 1/4,  
  colour = c('grey24')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("C9orf72 cases") +
scale_x_discrete(name = "Non cases", breaks=c("Case_WT"), labels=c("")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")


#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/2updated.BoxPlot_C9orf72cases.Merged.WT.png", width = 40, height = 20, dpi = 300, units = "cm", plot = p2) 

data = filter (data, rs113247976_T == "1" |  rs113247976_T == "0")

p <- ggplot(data, aes(x = as.factor(rs113247976_T), y=age_at_onset, fill= rs113247976_T)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('#CE4A7EFF', '#CE4A7EFF'), alpha = 1/4,  
  colour = c('#CE4A7EFF', '#CE4A7EFF')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("rs113247976 Genotype") +
scale_x_discrete(name = "rs113247976 Genotype", breaks=c("0","1"), labels=c("CC","CT")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")

#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/updated.BoxPlot_rs113247976.Merged.WT.png", width = 40, height = 20, dpi = 300, units = "cm", plot = p2) 

data = merge(raw, data3groups, by = "FID")
data = filter(data, diagnosis2 == "Case_WT")



p <- ggplot(data, aes(x = as.factor(rs9901522_T), y=age_at_onset, fill=rs9901522_T)) +
  geom_boxplot(width=0.6, fatten = 4, fill=c('#354E71', '#354E71', '#354E71'), alpha = 1/4,  colour = c('#354E71', '#354E71', '#354E71')) + 
  theme_classic(base_size = 40) + coord_flip()
  p2 = p + ylab("Age at onset") + xlab("rs9901522 Genotype") +
scale_x_discrete(name = "rs9901522 Genotype", breaks=c("0","1", "2"), labels=c("CC","CT", "TT")) + 
theme(plot.title = element_text(size = 40), legend.position = "none")


#ggsave(filename = "/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/Merged.toPLOT.LOCAL.2021-04-11.GRS/BoxPlot_rs9901522.Merged.WT.png", width = 40, height = 30, dpi = 300, units = "cm", plot = p2) 

profile= read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/Merged.162.RISK.profile", header = T)

profile$FID_IID = profile$FID
#PHENO FILE 
pheno = read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2021-03-26.MergingDatasets/PCs/UPDATED.COV.ALSplusALSFTD.C9.txt", header = T)




#merge pheno with profile
data = merge(profile,pheno, by= "FID_IID")


table(data$diagnosis3)
table(data$diagnosis2)
#Case_EXP, Case_WT, Control_EXP, Control_WT
table(data$diagnosis4)

dim(data)
data$PHENO = data$PHENO.x

# REGRESSION C9ORF72CARRIERS using GRS without C9orf72
data <- data %>% mutate (diagnosis5 = ifelse(diagnosis2 == "Case_EXP", "Case_EXP", 
                                                        ifelse(diagnosis2 == "Case_WT", "Case_WT",
                                                               ifelse(diagnosis3 == "Control", "Control","no"))))


data3groups = filter(data, diagnosis5 == "Case_EXP" |diagnosis5 == "Case_WT" | diagnosis5 == "Control")
dim(data3groups)

data3groups = filter(data3groups, age_at_onset > 1)
dim(data3groups)





# open raw data file
raw = fread("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Bin/162variants.Merged.Discovery.raw", header = T)
# open raw data file
data3groups$FID = data3groups$FID.x

data = merge(raw, data3groups, by = "FID")





data1 = data %>% group_by(diagnosis2, rs113247976_T) %>% 
  summarize(mean_AAO = mean(age_at_onset), 
            SD_AAO = sd(age_at_onset))
data1 = as.data.table(data1)
print(data1)

data2 = data %>% group_by(diagnosis2, rs9901522_T) %>% 
  summarize(mean_AAO = mean(age_at_onset), 
            SD_AAO = sd(age_at_onset))
data2 = as.data.table(data2)
print(data2)


    
data3 = data %>% group_by(diagnosis2) %>% 
  summarize(mean_AAO = mean(age_at_onset), 
            SD_AAO = sd(age_at_onset),
            n = n())
data3 = as.data.table(data3)
print(data3)
  
data3 = data %>% group_by(diagnosis3) %>% 
  summarize(mean_AAO = mean(age_at_onset), 
            SD_AAO = sd(age_at_onset),
            n = n())
data3 = as.data.table(data3)
print(data3)






[+] Loading gcc  9.2.0  ... 
[+] Loading GSL 2.6 for GCC 9.2.0 ...
[-] Unloading gcc  9.2.0  ... 
[+] Loading gcc  9.2.0  ... 
[+] Loading openmpi 3.1.4  for GCC 9.2.0 
[+] Loading ImageMagick  7.0.8  on cn0912 
[+] Loading HDF5  1.10.4 
[-] Unloading gcc  9.2.0  ... 
[+] Loading gcc  9.2.0  ... 
[+] Loading NetCDF 4.7.4_gcc9.2.0 
[+] Loading pandoc  2.17.1.1  on cn0912 
[+] Loading pcre2 10.21  ... 
[+] Loading R 4.0.0 



R version 4.0.0 (2020-04-24) -- "Arbor Day"
Copyright (C) 2020 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> require(dplyr)


Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
package ‘dplyr’ was built under R version 4.0.3 
Loading required package: tidyverse


> require(tidyverse)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.2.0     ✔ forcats 0.5.0
✔ readr   2.1.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning messages:
1: package ‘ggplot2’ was built under R version 4.0.3 
2: package ‘tibble’ was built under R version 4.0.3 
3: package ‘tidyr’ was built under R version 4.0.5 
4: package ‘readr’ was built under R version 4.0.3 


...

> require(RColorBrewer)
> setwd("Merged.toPLOT.LOCAL.2021-04-11.GRS")
> 
> 
> 
> ############################################# OPEN .PROFILE AND COVARIATES FILES ##################################################
> 
> profile= read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2020-12-2.GRS/Results/Merged.162.RISK.profile", header = T)
> profile$FID_IID = profile$FID
> #PHENO FILE 
> pheno = read.table("/data/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2021-03-26.MergingDatasets/PCs/UPDATED.COV.ALSplusALSFTD.C9.txt", header = T)
> dim(pheno)
[1] 42090    38
> 
> 
> 
> 
> 
> #merge pheno with profile
> data = merge(profile,pheno, by= "FID_IID")
> 
> 
> table(data$diagnosis3)

   Case Control 
   7854   34236 
> table(data$diagnosis2)

   Case_EXP     Case_WT Control_EXP  Control_WT 
        817        7037           1       34235 
> #Case_EXP, Case_WT, Control_EXP, Control_WT
> table(data$diagnosis4)

    ALS Control     FTD 
   7703   34236     151 
> 
> dim(data)
[1] 42090    44
> data$P

`summarise()` has grouped output by 'diagnosis2'. You can override using the `.groups` argument.


> data1 = as.data.table(data1)
> print(data1)
    diagnosis2 rs113247976_T mean_AAO    SD_AAO
1:    Case_EXP             0 57.80769  9.487317
2:    Case_EXP             1 54.18919 11.087625
3:     Case_WT             0 60.07588 12.535801
4:     Case_WT             1 60.06311 12.135049
5:     Case_WT             2 61.75000 11.757976
6: Control_EXP             0 82.00000        NA
7:  Control_WT             0 63.39280 13.795042
8:  Control_WT             1 63.77818 13.179143
9:  Control_WT             2 61.83333 12.890565
> 
> data2 = data %>% group_by(diagnosis2, rs9901522_T) %>% 
+   summarize(mean_AAO = mean(age_at_onset), 
+             SD_AAO = sd(age_at_onset))


`summarise()` has grouped output by 'diagnosis2'. You can override using the `.groups` argument.


> data2 = as.data.table(data2)
> print(data2)
     diagnosis2 rs9901522_T mean_AAO    SD_AAO
 1:    Case_EXP           0 58.02762  9.310195
 2:    Case_EXP           1 54.96629 11.219352
 3:    Case_EXP           2 47.75000  6.396614
 4:     Case_WT           0 60.04416 12.527851
 5:     Case_WT           1 60.31776 12.487447
 6:     Case_WT           2 59.72727 12.780870
 7: Control_EXP           0 82.00000        NA
 8:  Control_WT           0 63.41266 13.752461
 9:  Control_WT           1 63.29784 14.017007
10:  Control_WT           2 63.90909 13.387183
> 
> 
>     
> data3 = data %>% group_by(diagnosis2) %>% 
+   summarize(mean_AAO = mean(age_at_onset), 
+             SD_AAO = sd(age_at_onset),
+             n = n())
> data3 = as.data.table(data3)
> print(data3)
    diagnosis2 mean_AAO    SD_AAO     n
1:    Case_EXP 57.64382  9.587404   817
2:     Case_WT 60.07645 12.522265  7037
3: Control_EXP 82.00000        NA     1
4:  Control_WT 63.40181 13.780152 34235
>   
> data3 = data %>%

In [14]:
%%bash
cd /gpfs/gsfs11/users/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2022_01_14.ManuscriptRepo

echo "rs113247976	C9orf72 status	mean AAO	SD AAO	Counts
CC	carrier	57.81	9.49	780
CT	carrier	54.19	11.09	37
TT	carrier	NA	NA	0
CC	non-carrier	60.08	12.54	6827
CT	non-carrier	60.06	12.14	206
TT	non-carrier	61.75	11.76	4" > rs113247976.table.tab

echo "rs9901522	C9orf72 status	mean AAO	SD AAO	Counts
CC	carrier	58.02762	9.310195	724
CT	carrier	54.96629	11.219352	89
TT	carrier	47.75	6.396614	4
CC	non-carrier	60.04416	12.527851	6159
CT	non-carrier	60.31776	12.487447	856
TT	non-carrier	59.72727	12.78087	22" > rs9901522.table.tab


echo "Model	Variant	C9orf72 status	beta	p-value	F-statistics	p-value F
Additive	rs113247976	carrier	-3.360	0.038	1.554	0.050
Dominant	rs113247976	carrier	-3.360	0.038	1.554	0.050
Additive	rs9901522_T	carrier	-3.513	4.05E-04	1.946	5.90E-03
Dominant	rs9901522_T	carrier	-3.488	9.76E-04	1.867	9.31E-03
Additive	rs113247976	non-carrier	0.157	0.715	10.210	< 2.2e-16
Dominant	rs113247976	non-carrier	0.125	0.88547	10.210	< 2.2e-16
Additive	rs9901522_T	non-carrier	0.146	0.863	10.210	< 2.2e-16
Dominant	rs9901522_T	non-carrier	0.1845	0.67926	10.210	< 2.2e-16" > Dom.Add.Models.tab

In [12]:
import numpy as np
import pandas as pd
res = pd.read_csv("/gpfs/gsfs11/users/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2022_01_14.ManuscriptRepo/rs113247976.table.tab",sep="\t")
res



,rs113247976,C9orf72 status,mean AAO,SD AAO,Counts
0,CC,carrier,57.81,9.49,780
1,CT,carrier,54.19,11.09,37
2,TT,carrier,NaN,NaN,0
3,CC,non-carrier,60.08,12.54,6827
4,CT,non-carrier,60.06,12.14,206
5,TT,non-carrier,61.75,11.76,4


In [13]:
import numpy as np
import pandas as pd
res = pd.read_csv("/gpfs/gsfs11/users/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2022_01_14.ManuscriptRepo/rs9901522.table.tab",sep="\t")
res



,rs9901522,C9orf72 status,mean AAO,SD AAO,Counts
0,CC,carrier,58.02762,9.310195,724
1,CT,carrier,54.96629,11.219352,89
2,TT,carrier,47.75000,6.396614,4
3,CC,non-carrier,60.04416,12.527851,6159
4,CT,non-carrier,60.31776,12.487447,856
5,TT,non-carrier,59.72727,12.780870,22


In [15]:
import numpy as np
import pandas as pd
res = pd.read_csv("/gpfs/gsfs11/users/ALS_50k/SaraSaez_ALS/PROJECT2_C9_AAO/2022_01_14.ManuscriptRepo/Dom.Add.Models.tab",sep="\t")
res


,Model,Variant,C9orf72 status,beta,p-value,F-statistics,p-value F
0,Additive,rs113247976,carrier,-3.3600,0.038000,1.554,0.050
1,Dominant,rs113247976,carrier,-3.3600,0.038000,1.554,0.050
2,Additive,rs9901522_T,carrier,-3.5130,0.000405,1.946,5.90E-03
3,Dominant,rs9901522_T,carrier,-3.4880,0.000976,1.867,9.31E-03
4,Additive,rs113247976,non-carrier,0.1570,0.715000,10.210,< 2.2e-16
5,Dominant,rs113247976,non-carrier,0.1250,0.885470,10.210,< 2.2e-16
6,Additive,rs9901522_T,non-carrier,0.1460,0.863000,10.210,< 2.2e-16
7,Dominant,rs9901522_T,non-carrier,0.1845,0.679260,10.210,< 2.2e-16
